In [2]:
import requests
import io
import numpy as np
import os

#Loads data from url
def make_request(url):
    print("Requesting data from {}...".format(url))
    response = requests.get('https://content.codecademy.com/courses/deeplearning-with-tensorflow/'+url)
    response.raise_for_status()
    response_data = io.BytesIO(response.content)
    return response_data
    
#Loads galaxy data
def load_galaxy_data():
  
  #If cached file not found, loads data from url
    if not os.path.isfile('./cached_data.npz'):
        response_data = make_request(url='galaxydata.npz')

    with open("cached_data.npz","wb") as save_file:
        save_file.write(response_data.read())
 
  #Load data using NumPy
    data = np.load('cached_data.npz')

    print("Successfully loaded galaxy data!")
  
    return data["data"],data["labels"]

In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import train_test_split

In [4]:
input_data, labels = load_galaxy_data()

Requesting data from galaxydata.npz...
Successfully loaded galaxy data!


In [5]:
#preprocess input
data_generator = ImageDataGenerator(rescale=1./255)

In [6]:
input_data.shape

(1400, 128, 128, 3)

In [7]:
labels.shape

(1400, 4)

In [8]:
x_train, x_valid, y_train, y_valid = train_test_split(input_data, 
                                                      labels, test_size=0.20, 
                                                      stratify=labels, 
                                                      shuffle=True, random_state=222)


In [9]:
training_iterator = data_generator.flow(x_train, y_train,batch_size=5)
validation_iterator = data_generator.flow(x_valid, y_valid, batch_size=5)


In [15]:
#build model
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(128, 128, 3)))
model.add(tf.keras.layers.Conv2D(8, 3, strides=2, activation="relu")) 
model.add(tf.keras.layers.MaxPooling2D(
    pool_size=(2, 2), strides=(2,2)))
model.add(tf.keras.layers.Conv2D(8, 3, strides=2, activation="relu")) 
model.add(tf.keras.layers.MaxPooling2D(
    pool_size=(2,2), strides=(2,2)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(16, activation="relu"))
model.add(tf.keras.layers.Dense(4, activation="softmax"))

In [16]:
#compile
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=[tf.keras.metrics.CategoricalAccuracy(),tf.keras.metrics.AUC()])

In [17]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 63, 63, 8)         224       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 31, 31, 8)        0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 15, 15, 8)         584       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 7, 7, 8)          0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 392)               0         
                                                                 
 dense_3 (Dense)             (None, 16)               

In [18]:
#train
model.fit(
        training_iterator,
        steps_per_epoch=len(x_train)/5,
        epochs=8,
        validation_data=validation_iterator,
        validation_steps=len(x_valid)/5)

Epoch 1/8
224/224 [==============================] - 2s 6ms/step - loss: 1.3133 - categorical_accuracy: 0.3482 - auc_1: 0.6404 - val_loss: 1.1263 - val_categorical_accuracy: 0.5036 - val_auc_1: 0.7791
Epoch 2/8
224/224 [==============================] - 1s 5ms/step - loss: 1.1586 - categorical_accuracy: 0.5089 - auc_1: 0.7533 - val_loss: 1.2185 - val_categorical_accuracy: 0.4571 - val_auc_1: 0.7206
Epoch 3/8
224/224 [==============================] - 1s 5ms/step - loss: 1.1027 - categorical_accuracy: 0.5527 - auc_1: 0.7808 - val_loss: 1.0189 - val_categorical_accuracy: 0.6000 - val_auc_1: 0.8242
Epoch 4/8
224/224 [==============================] - 1s 5ms/step - loss: 1.0229 - categorical_accuracy: 0.5804 - auc_1: 0.8155 - val_loss: 0.9154 - val_categorical_accuracy: 0.6464 - val_auc_1: 0.8633
Epoch 5/8
224/224 [==============================] - 1s 5ms/step - loss: 0.9547 - categorical_accuracy: 0.6214 - auc_1: 0.8409 - val_loss: 0.8571 - val_categorical_accuracy: 0.6929 - val_auc_1: 0.